### And welcome to Week 4, Day 3 - more LangGraph..

In [ ]:
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain_openai import ChatOpenAI
from typing import TypedDict


In [ ]:
# Our favorite first step! Crew was doing this for us, by the way.
load_dotenv(override=True)


### First, let's go set up LangSmith!

https://langsmith.com

### Next, here is a useful function in LangChain community:

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper

serper = GoogleSerperAPIWrapper()
serper.run("What is the capital of France?")

### Now here is a LangChain wrapper class for converting functions into Tools

In [ ]:
from langchain.agents import Tool

tool_search =Tool(
        name="search",
        func=serper.run,
        description="Useful for when you need more information from an online search"
    )



### Now we can try out the tool the langchain way

In [ ]:
tool_search.invoke("What is the capital of France?")

### And now let's write a tool ourselves

We'll pick a familiar one

In [ ]:
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_user = os.getenv("PUSHOVER_USER")
pushover_url = "https://api.pushover.net/1/messages.json"

def push(text: str):
    """Send a push notification to the user"""
    requests.post(pushover_url, data = {"token": pushover_token, "user": pushover_user, "message": text})

In [ ]:
tool_push = Tool(
        name="send_push_notification",
        func=push,
        description="useful for when you want to send a push notification"
    )

tool_push.invoke("Hello, me")

### Back to the Graph from yesterday

One small change - using TypedDict instead of BaseModel for the State object

When we implement tools, we always need to make 2 changes to the code:

1. Changes to provide the tools to OpenAI in json when we make the call

2. Changes to handle the results back: look for the model staying that the finish_reason=="tool_calls" and then retrieve the call, run the function, provide the results.

### Bring them together

In [ ]:
tools = [tool_search, tool_push]

In [ ]:
# Step 1: Define the State object
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
# Step 2: Start the Graph Builder with this State class
graph_builder = StateGraph(State)

In [ ]:
# This is different:

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

In [ ]:
# Step 3: Create a Node


def chatbot(state: State):
    # handling the request to the LLM and packaging up the tools JSON
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
#  handling what happens if a if there is a tool request, it's handling the unbundling of the request and the actual calling of the tool function.
graph_builder.add_node("tools", ToolNode(tools=tools))

In [ ]:
# Step 4: Create Edges

# chatbot -> tools, undersome condition
# We only want to call those tools if the model has returned a response that the finish reason is tool calls.
# Not normal edge, but a conditional edge, it's an edge that's only triggered in certain conditions.
# And that condition called tools condition is again pre-canned with Landgraf.
# And of course it looks to see if the finish reason is handle tools.
# The finish reason is tool calls.
# Tooling calling is just an if statement.
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")

# Any time a tool is called, we return to the chatbot to decide the next step
# Also have to make an edge between tools. Back to chatbot again, because the result of this, the result of running the tool, the output needs to get fed back into the chatbot and it needs to continue processing from there.
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

In [ ]:
# Step 5: Compile the Graph
# This node is a canned node that's responsible for calling the tools that are relevant. And then the results comes back.
# And this is a solid line because if it's got here it should always come back.
# And this is a dotted line because only in the event that this didn't happen then it will come to end naturally.
# Landgraf automatically adds an end node for any any unresolved condition like that.
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

### That's it! And, let's do this:

In [ ]:
# Invoke grpah
def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]})
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

## OK it's time to add Memory!

### BUT WAIT!

We have this whole Graph maintaining the state and appending to the state.

Why isn't this handling memory?

### This is a crucial point for understanding LangGraph

> A super-step can be considered a single iteration over the graph nodes. Nodes that run in parallel are part of the same super-step, while nodes that run sequentially belong to separate super-steps.


One "Super-Step" of the graph represents one invocation of passing messages between agents.

In idomatic LangGraph, you call invoke to run your graph for each super-step; for each interaction.

The reducer handles state updates automatically within one super-step, but not between them.

That is what checkpointing achieves.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# Recreating a new memory object resets everything
memory = MemorySaver()

In [ ]:
# Steps 1 and 2
graph_builder = StateGraph(State)


# Step 3
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    print(state)
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

# Step 4
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# Step 5
graph = graph_builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# It's a dictionary with one field configurable, and then that field you have to put in a thread ID and
# thread ID doesn't mean like a technical thread, it's meant to be like a conversation thread, like
# it's something that refers to this, this thread of memories that needs to be connected together.
# And so that's how you specify that, that this is this particular grouping in memory.
# changing the thread ID also changes the memory that is being referenced.
config = {"configurable": {"thread_id": "1"}}

# It's it's relatively lightweight and simple, but very powerful.
# And that's a nice combination that I can get behind.
# And you can come in and look of course at the different checkpoints for either thread number one or
# thread number two.

#And then when you invoke the graph you have to pass in that config.
# That's how you make sure that when you're invoking the graph, it's being associated with the right
# sort of slot in memory.
def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

In [ ]:
# Given that the the config and we get back this thing called a state snapshot, and this has in it the messages and then the complete history the conversation so far.
graph.get_state(config)

In [ ]:
# Most recent first
# And what we get now is each step in time, every super step, every time we invoke the graph, the complete, uh, snapshot at that moment, starting from most recent at the top and then going back in time.
list(graph.get_state_history(config))

# Langshan allows you to step back in time to any prior moment.
# When you're passing in that config, the configurable thread ID you can pass in a checkpoint ID to kind
# of rewind to a previous moment, and then replay that through the graph.
# Uh, and this, this gives you this ability to basically do what they call a time travel.
# That's the Landgraf official name for it, which is really to be able to move back, get your snapshot
# at any point in time and be able to rerun it from there.
# And this is this is really great because it allows you to build systems that are repeatable and robust.
# If something falls over, you can restart it from any snapshot, any point in time.
# And you've got this kind of full tracking on everything that happened.
# So yeah, it's it's it's simple but it's elegant.
# And this is a case of, of an abstraction that really makes sense to me.

### LangGraph gives you tools to set the state back to a prior point in time, to branch off:

```
config = {"configurable": {"thread_id": "1", "checkpoint_id": ...}}
graph.invoke(None, config=config)
```

And this allows you to build stable systems that can be recovered and rerun from any prior checkpoint.

### And now let's store in SQL

### And this is the power of LangGraph.

In [ ]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

db_path = "memory.db"
conn = sqlite3.connect(db_path, check_same_thread=False)
sql_memory = SqliteSaver(conn)

In [ ]:
# Steps 1 and 2
graph_builder = StateGraph(State)


# Step 3
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    print(state)
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

# Step 4
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# Step 5
graph = graph_builder.compile(checkpointer=sql_memory)
display(Image(graph.get_graph().draw_mermaid_png()))
 

In [ ]:
# Uh, completely bring everything fresh.
# And of course, it knows who I am.
# And it knows that because you can see right here in this directory, there's a bunch of database objects.
# It's storing it in a SQLite database.
# And so we have persistent memory with changing like one word uh from, from my code.

config = {"configurable": {"thread_id": "3"}}

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()